In [1]:
%load_ext autoreload
%autoreload 2
from datetime import date, datetime, timedelta

import os
import sys
sys.path.append('..')

import lzma
import pickle

import numpy as np
import pandas as pd

from bidder import db

from bidder.wb_data import DataFetcher, process, get_campaigns
from bidder.wb_ad_campaigns import process_wb_ad_campaigns
from bidder.daily_orders import process_daily_orders
from bidder.wb_bidder import Bidder
from bidder import utils
from bidder.models.cr_view_v1 import CrViewModelV1
from bidder.models.analytics_baseline import AnalyticsBaselineV1

import plotly.express as px

pd.options.display.max_rows = 200
pd.options.display.max_columns = None

In [2]:
dt_s = '250615T070000'

with lzma.open(f'./data/raw_data_304785544_{dt_s}.lzma', 'rb') as fo:
    data_fetcher = pickle.load(fo)

processed = process(data_fetcher)

df = processed.processed_wb_ad_campaigns.df
df = df[['dt', 'nm_id', 'advert_id', 'hour_of_week', 'hour', 'cpm_1', 'cr_view', 'ctr', 'cr_click']].reset_index(drop=True)
df.rename(columns={'dt': 'calculation_dt'}, inplace=True)
df.dropna(subset=['cpm_1'], inplace=True)
calculations_df = df

calculations_df['calculation_dt'].max()

Timestamp('2025-06-15 06:00:00')

In [3]:
def select(df, nm_id = '304785544', advert_id = set(['25438064', '22499904'])):
    if advert_id is None or len(advert_id) == 0:
        return df[df['nm_id'] == nm_id]
    else:
        return df[(df['nm_id'] == nm_id) & (df['advert_id'].isin(advert_id))]

df = select(df, advert_id = set(['25438064']))
df['cpm_1'].describe()

count    274.000000
mean       3.706518
std        0.378264
min        2.166667
25%        3.633375
50%        3.811511
75%        3.953418
max        4.407721
Name: cpm_1, dtype: float64

In [4]:
df = select(processed.processed_wb_ad_campaigns.df)
df[df['hour_orders'] > 0].sort_values('cr_view')

,dt,nm_id,advert_id,legal_entity,product_id,mp_id,category,week_day,week_start,date,hour_of_week,hour,hour_views,hour_clicks,hour_orders,hour_atbs,hour_sum,cpm_1,cpm_1_round,cr_view,ctr,cr_click
42670,2025-05-17 23:00:00,304785544,22499904,ИНТЕР,DRL2-1,304785544.0,Инструменты,5,2025-05-12,2025-05-17,143,23.0,2071.0,42.0,1.0,7.0,969.67,0.468213,0.47,0.000483,0.020280,0.023810
42671,2025-05-17 23:00:00,304785544,25438064,ИНТЕР,DRL2-1,304785544.0,Инструменты,5,2025-05-12,2025-05-17,143,23.0,1726.0,83.0,1.0,9.0,6451.50,3.737833,3.74,0.000579,0.048088,0.012048
42758,2025-05-18 23:00:00,304785544,22499904,ИНТЕР,DRL2-1,304785544.0,Инструменты,6,2025-05-12,2025-05-18,167,23.0,1440.0,52.0,1.0,10.0,674.18,0.468181,0.47,0.000694,0.036111,0.019231
68303,2025-06-04 09:00:00,304785544,25438064,ИНТЕР,DRL2-1,304785544.0,Инструменты,2,2025-06-02,2025-06-04,57,9.0,736.0,46.0,1.0,4.0,2875.18,3.906495,3.91,0.001359,0.062500,0.021739
42921,2025-05-19 09:00:00,304785544,25438064,ИНТЕР,DRL2-1,304785544.0,Инструменты,0,2025-05-19,2025-05-19,9,9.0,609.0,37.0,1.0,7.0,2255.77,3.704056,3.70,0.001642,0.060755,0.027027
60946,2025-05-31 10:00:00,304785544,25438064,ИНТЕР,DRL2-1,304785544.0,Инструменты,5,2025-05-26,2025-05-31,130,10.0,414.0,17.0,1.0,2.0,1584.86,3.828164,3.83,0.002415,0.041063,0.058824
66487,2025-06-03 09:00:00,304785544,25438064,ИНТЕР,DRL2-1,304785544.0,Инструменты,1,2025-06-02,2025-06-03,33,9.0,378.0,37.0,1.0,2.0,1458.78,3.859206,3.86,0.002646,0.097884,0.027027
48560,2025-05-22 08:00:00,304785544,25438064,ИНТЕР,DRL2-1,304785544.0,Инструменты,3,2025-05-19,2025-05-22,80,8.0,365.0,19.0,1.0,1.0,1374.87,3.766767,3.77,0.002740,0.052055,0.052632
52250,2025-05-26 11:00:00,304785544,22499904,ИНТЕР,DRL2-1,304785544.0,Инструменты,0,2025-05-26,2025-05-26,11,11.0,357.0,20.0,1.0,3.0,65.64,0.183866,0.18,0.002801,0.056022,0.050000
44708,2025-05-20 09:00:00,304785544,25438064,ИНТЕР,DRL2-1,304785544.0,Инструменты,1,2025-05-19,2025-05-20,33,9.0,353.0,10.0,1.0,1.0,1336.10,3.784986,3.78,0.002833,0.028329,0.100000


In [5]:
processed.campaigns_df

,nm_id,advert_id,cost_in_rub,seller_id,mp_id,category,legal_entity,marketplace_name,product_id,cost_price_with_sales_fee_to_cost_price,cost_price_with_sales_fee,price_in_rub
0,304785544,25438064,1232.0,DRL2-1,304785544.0,Инструменты,ИНТЕР,Wildberries,DRL2-1,1.507648,1857.422932,2000.0


In [6]:
cpm_1000 = 2000-1857.422932
cpm_1000

142.57706800000005

In [7]:
3.82 / (2000 - 1857.422932)

0.026792527392974573

In [10]:
df.groupby(['hour'])['cr_view'].max()

hour
0.0     0.000000
1.0     0.000000
2.0     0.000000
3.0     0.000000
4.0     0.000000
5.0     0.000000
6.0     0.011236
7.0     0.003876
8.0     0.005155
9.0     0.066667
10.0    0.012048
11.0    0.200000
12.0    0.500000
13.0    0.166667
14.0    0.000000
15.0    0.004065
16.0    0.000000
17.0    0.035714
18.0    1.000000
19.0    0.015385
20.0    0.015625
21.0    0.000000
22.0    0.017544
23.0    0.040000
Name: cr_view, dtype: float64